<a href="https://colab.research.google.com/github/hayleypc/HawaiiClimate/blob/main/expertsocmodel_summaries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from google.colab import drive

In [6]:
from shapely.geometry import Point

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


HPC to pull out more specific info for minerals and C, considering 75th quartile for mineral pools, etc. Create new table, w/ timestamp Have lots of NAs that she will be matching w/ the geospatial data All 0-20 cm, rest of data is 0-15 cm Need break down by land-use (code currently hinges on land-use having similar inputs across certain soil types). Land use & minerals always connected Which quartiles to use? Could bootstrap and figure out where it destabilizes - will run w/ 90th for now but put a pin in it for when we start putting together manuscript HPC to pull NPP from GEE, can also use lit values 2 data sources to look at for NPP and still needs to match w/ land-use (HiClimate data portal NDVI - going to look into that)

pulled the following data preprocessing steps from HI_NN script: https://colab.research.google.com/drive/1Rzl-J5F2KIQqk-RWg-pBg2dP8B_M28H6?authuser=1#scrollTo=Quz3PwWnrs-o

In [3]:
gdf = gpd.read_file('/content/drive/MyDrive/hawaii_soils/HI soils data/2024_update/annotated_combo_imputed_SOC.gpkg')

In [4]:
drivers_gpd = gpd.read_file('/content/drive/MyDrive/hawaii_soils/Analysis Data/250_summary_grid_dt.gpkg')
soils_csv = pd.read_csv('/content/drive/MyDrive/hawaii_soils/HI soils data/combined_soc_2024_04_05.csv')

In [7]:
# Preprocess soils data
soils_csv = soils_csv.dropna(subset=['latitude', 'longitude'])
soils_csv['geometry'] = soils_csv.apply(lambda row: Point(float(row['longitude']), float(row['latitude'])), axis=1)
soils_gpd = gpd.GeoDataFrame(soils_csv, geometry='geometry', crs="EPSG:4326")

# Merge and transform GeoDataFrames
soils_gpd = pd.merge(soils_gpd, gdf[['dist_id', 'unique_id']], on='unique_id', how='inner')
soils_gpd = soils_gpd.to_crs(drivers_gpd.crs)

# Perform spatial join
matched_data = gpd.sjoin_nearest(soils_gpd, drivers_gpd, how='left', distance_col='distance')

/usr/local/lib/python3.10/dist-packages/geopandas/array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [8]:
# Additional preprocessing steps
drivers_gpd['x_driver'] = drivers_gpd.geometry.x
drivers_gpd['y_driver'] = drivers_gpd.geometry.y
soils_gpd['x_sample'] = soils_gpd.geometry.x
soils_gpd['y_sample'] = soils_gpd.geometry.y

In [10]:
# Reproject both GeoDataFrames to the same CRS
soils_buffered = soils_gpd.to_crs(epsg=32604)
drivers_gpd = drivers_gpd.to_crs(soils_buffered.crs)

# Apply buffer in the same CRS
soils_buffered['x_sample'] = soils_buffered.geometry.x
soils_buffered['y_sample'] = soils_buffered.geometry.y
soils_buffered.geometry = soils_buffered.geometry.buffer(1000)

# Perform spatial join
matches_within_distance = gpd.sjoin(soils_buffered, drivers_gpd, how='left', predicate='intersects')

In [11]:
soils_gpd.columns

Index(['source_dataset', 'island', 'soil_column_id', 'unique_id', 'depth_top',
       'depth_bottom', 'depth_adj_bottom', 'latitude', 'longitude', 'c_perc',
       'imp_c', 'bulk_density', 'imp_bd', 'hole_id', 'geometry', 'dist_id',
       'x_sample', 'y_sample'],
      dtype='object')